# Construção da base de notícias ESG a partir da captura do Google RSS

- Autor: Daniel Saraiva Leite - 2023
- Projeto Análise de sentimentos sobre notícias do tema ESG

In [1]:
import warnings
import pandas as pd
import datetime as dt
from noticias_google_buscador import busca_noticias_google_news
from noticias_google_buscador_esg import *
import numpy as np
from noticias_io import *

warnings.filterwarnings('ignore')

/Users/danielsaraivaleite/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/danielsaraivaleite/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/Users/danielsaraivaleite/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[nltk_data] Downloading package rslp to
[nltk_data]     /Users/danielsaraivaleite/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Us

In [2]:
df_atual = le_base_noticias_bruta_para_df()
df_empresas = le_lista_empresas_para_df()

hash_noticias_ja_existentes = list(df_atual['hash'])

# processa empresas
for i, row in df_empresas[df_empresas['buscar'].str.upper() == 'SIM'].iterrows():
    
    empresa = row['empresa']
    
    aplicar_filtros= ( (not pd.isnull(row['aplicar_filtros_relevancia'])) and   row['aplicar_filtros_relevancia'].upper() == 'SIM')
    inclui_apenas_nome_empresa = ( (not pd.isnull(row['tipo_busca_exaustiva'])) and    row['tipo_busca_exaustiva'].upper() == 'SIM')
    
    atualiza = (  (not pd.isnull(row['tipo_busca_atualizacao'])) and    row['tipo_busca_atualizacao'].upper() == 'SIM' and (not pd.isnull(row['ultima_data_publicacao'])))
    ultima_data = row['ultima_data_publicacao'].date()
    
    print('\n' +  str(dt.datetime.now())  + ' Buscando empresa ' + empresa)
    dfEmpresasListadas = df_empresas
    
    df1 = busca_noticias_google_esg(empresa, atualiza=atualiza, ultima_data = ultima_data, inclui_apenas_nome_empresa = inclui_apenas_nome_empresa)
    
    if len(df1) >0:
        # ajustes
        #df1['data_publicacao'] = df1['data_publicacao'].dt.date
        df1 = ajusta_nomes_empresas_dataframe(df1)

        # cria hash
        df1['hash'] = df1.apply(lambda row: criar_hash_noticia(texto=row['titulo'], empresa=row['empresa'], titulo=row['titulo'], data=row['data_publicacao']), axis=1)
        print(str(dt.datetime.now())  + ' ... noticias encontradas: ' + str(len(df1)))
    
    if len(df1) >0:
        df1 = df1[~df1['hash'].isin(hash_noticias_ja_existentes)]
        print(str(dt.datetime.now())  + ' ... noticias novas: ' + str(len(df1)))
    
    if len(df1) > 0:
        df2 = recupera_noticias_completas(df1)
        #print(df1)
        print(str(dt.datetime.now())  + ' ... textos buscados: ' + str(len(df2)))
            
        if aplicar_filtros:
            df3 = filtra_noticias_nao_relacionadas(df2, empresa)
            if len(df3) >0:
                df4 = filtra_citacao_relevante(df3, empresa, dfEmpresasListadas )
                print(str(dt.datetime.now())  + ' ... novas noticias após filtro citaçoes: ' + str(len(df4)))
                
        else:
            df4 = df2
        
        if len(df4) >0:
            df5 = classifica_textos_coletados(df4)
            df6 = df5
            print(str(dt.datetime.now())  + ' ... novas noticias após filtros: ' + str(len(df6)))
            df_atual = le_base_noticias_bruta_para_df()
            df = pd.concat([df_atual,df6]).drop_duplicates(ignore_index=True)
            df = df.drop_duplicates(['titulo','empresa','fonte', 'texto_completo'], keep='first')
            salva_base_noticias_bruta(df)


# complementando os dados
# atualiza nome, cnpj, setor
df_noticias = le_base_noticias_bruta_para_df()
df_empresas = le_lista_empresas_para_df()
df_noticias['Nome'] = df_noticias['empresa'].map(df_empresas.set_index('empresa')['Nome'])
df_noticias['CNPJ'] = df_noticias['empresa'].map(df_empresas.set_index('empresa')['CNPJ'])
df_noticias['Razão social'] = df_noticias['empresa'].map(df_empresas.set_index('empresa')['RAZÃO SOCIAL'])
df_noticias['Código'] = df_noticias['empresa'].map(df_empresas.set_index('empresa')['Código'])
df_noticias['Setor'] = df_noticias['empresa'].map(df_empresas.set_index('empresa')['SETOR'])
salva_base_noticias_bruta(df_noticias)

# atualiza ultima data de captura
df_empresas['ultima_data_publicacao'] = df_empresas['empresa'].map(df_noticias[['empresa', 'data_publicacao']].groupby('empresa').max()['data_publicacao'] )
salva_lista_empresas(df_empresas)

print('\nCaptura concluída')


2024-03-03 22:06:18.398006 Buscando empresa slc agricola
...... buscando no google slc+agricola
2024-03-03 22:06:57.576284 ... noticias encontradas: 37
2024-03-03 22:06:57.589154 ... noticias novas: 22
2024-03-03 22:07:40.296447 ... textos buscados: 18
2024-03-03 22:07:42.931477 ... novas noticias após filtros: 18

2024-03-03 22:07:44.264360 Buscando empresa taurus
...... buscando no google taurus
2024-03-03 22:08:23.801470 ... noticias encontradas: 44
2024-03-03 22:08:23.808762 ... noticias novas: 44
2024-03-03 22:09:32.609276 ... textos buscados: 44
2024-03-03 22:09:38.407759 ... novas noticias após filtros: 44

2024-03-03 22:09:39.717227 Buscando empresa terra santa propriedades agricolas
...... buscando no google terra+santa+propriedades+agricolas
2024-03-03 22:10:20.019581 ... noticias encontradas: 12
2024-03-03 22:10:20.028467 ... noticias novas: 11
2024-03-03 22:10:39.251429 ... textos buscados: 11
2024-03-03 22:10:40.693140 ... novas noticias após filtros: 11

2024-03-03 22:10